In [1]:
import numpy as np
import pandas as pd 
import scipy.sparse

In [2]:
import datetime
from datetime import date
from datetime import timedelta

In [3]:
from sklearn import datasets, linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [4]:
import pickle

train_data = pickle.load(open("train_data_with_feats.p","rb"))
test_data = pickle.load(open("test_data_with_feats.p","rb"))

In [5]:
val_set_rule = (train_data["sale_month"]==33)
train_set_rule = ((train_data["sale_month"]!=33)) # & (train_data["sale_month"]>2))
xtrain = (train_data.loc[train_set_rule,:]).iloc[:,6:]
#print(xtrain.head())
ytrain = (train_data.loc[train_set_rule,:]).iloc[:,5]
#print(ytrain.head())

xval = (train_data.loc[val_set_rule,:]).iloc[:,6:]
#print(xval.head())
yval = (train_data.loc[val_set_rule,:]).iloc[:,5]
#print(yval.head())


In [6]:
# Create linear regression object
regr = DecisionTreeRegressor(max_depth=20)

# Train the model using the training sets
regr.fit(xtrain, ytrain)

DecisionTreeRegressor(criterion='mse', max_depth=20, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [7]:
print("training MSE: {%f}" % mean_squared_error(ytrain,regr.predict(xtrain)))
print("validation MSE: {%f}" % mean_squared_error(yval,regr.predict(xval)))

training MSE: {0.005392}
validation MSE: {0.011067}


## Original
training MSE: {0.032447}
validation MSE: {0.078754}

## missing + log
training MSE: {0.005783}
validation MSE: {0.010810}

In [8]:
cols = train_data.columns
f_imp = dict()
for i in range(len(regr.feature_importances_)):
    f_imp[cols[i+6]]=regr.feature_importances_[i]

for k in f_imp.keys():
    if(f_imp[k]>0.05):
        print("%s -> %f"%(k,f_imp[k]))

item_id_shop_id_mon_9_rev_day_mon_9 -> 0.073147
item_id_mon_10_rev_day_mon_10 -> 0.189095
item_category_id_mon_4_rev_day_mon_4 -> 0.097511
item_category_id_mon_5_rev_day_mon_5 -> 0.069121


In [9]:
from xgboost import XGBClassifier
from xgboost import XGBRegressor

In [12]:
regr_2 = XGBRegressor(n_estimators=100,max_depth=3)
regr_2.fit(xtrain, ytrain)

print("training MSE: {%f}" % mean_squared_error(ytrain,regr_2.predict(xtrain)))
print("validation MSE: {%f}" % mean_squared_error(yval,regr_2.predict(xval)))

training MSE: {0.006713}
validation MSE: {0.006634}


In [13]:
test_data["pred"] = regr.predict(test_data.iloc[:,7:])

test_data["item_cnt_month"]= np.clip(test_data["pred"],0,20)
test_data.head()

,ID,shop_id,item_id,item_category_id,year,sale_month,true_rev_per_day,item_id_shop_id_mon_1_rev_day_mon_1,item_id_shop_id_mon_2_rev_day_mon_2,item_id_shop_id_mon_3_rev_day_mon_3,...,item_category_id_mon_5_rev_day_mon_5,item_category_id_mon_6_rev_day_mon_6,item_category_id_mon_7_rev_day_mon_7,item_category_id_mon_8_rev_day_mon_8,item_category_id_mon_9_rev_day_mon_9,item_category_id_mon_10_rev_day_mon_10,item_category_id_mon_11_rev_day_mon_11,item_category_id_mon_12_rev_day_mon_12,pred,item_cnt_month
0,0,5,5037,19,2015,34,0.0,0.03279,0.03279,0.03279,...,5.325284,5.325284,5.325284,5.325284,5.325284,5.325284,5.325284,5.325284,0.046581,0.046581
1,1,5,5320,55,2015,34,0.0,0.07058,0.07058,0.07058,...,5.792912,5.792912,5.792912,5.792912,5.792912,5.792912,5.792912,5.792912,0.045125,0.045125
2,2,5,5233,19,2015,34,0.0,0.07058,0.07058,0.07058,...,5.325284,5.325284,5.325284,5.325284,5.325284,5.325284,5.325284,5.325284,0.042168,0.042168
3,3,5,5232,23,2015,34,0.0,0.07058,0.07058,0.07058,...,5.175208,5.175208,5.175208,5.175208,5.175208,5.175208,5.175208,5.175208,0.069512,0.069512
4,4,5,5268,20,2015,34,0.0,0.07058,0.07058,0.07058,...,6.060991,6.060991,6.060991,6.060991,6.060991,6.060991,6.060991,6.060991,0.034718,0.034718


In [14]:
sub = test_data.loc[:,["ID","item_cnt_month"]]
sub.to_csv("data/sub_tree.csv",index=False)

In [15]:
test_data["pred2"] = regr_2.predict(test_data.iloc[:,7:-2])

test_data["item_cnt_month2"]= np.clip(test_data["pred2"],0,20)
test_data.head()


,ID,shop_id,item_id,item_category_id,year,sale_month,true_rev_per_day,item_id_shop_id_mon_1_rev_day_mon_1,item_id_shop_id_mon_2_rev_day_mon_2,item_id_shop_id_mon_3_rev_day_mon_3,...,item_category_id_mon_7_rev_day_mon_7,item_category_id_mon_8_rev_day_mon_8,item_category_id_mon_9_rev_day_mon_9,item_category_id_mon_10_rev_day_mon_10,item_category_id_mon_11_rev_day_mon_11,item_category_id_mon_12_rev_day_mon_12,pred,item_cnt_month,pred2,item_cnt_month2
0,0,5,5037,19,2015,34,0.0,0.03279,0.03279,0.03279,...,5.325284,5.325284,5.325284,5.325284,5.325284,5.325284,0.046581,0.046581,0.044759,0.044759
1,1,5,5320,55,2015,34,0.0,0.07058,0.07058,0.07058,...,5.792912,5.792912,5.792912,5.792912,5.792912,5.792912,0.045125,0.045125,0.052722,0.052722
2,2,5,5233,19,2015,34,0.0,0.07058,0.07058,0.07058,...,5.325284,5.325284,5.325284,5.325284,5.325284,5.325284,0.042168,0.042168,0.053453,0.053453
3,3,5,5232,23,2015,34,0.0,0.07058,0.07058,0.07058,...,5.175208,5.175208,5.175208,5.175208,5.175208,5.175208,0.069512,0.069512,0.303606,0.303606
4,4,5,5268,20,2015,34,0.0,0.07058,0.07058,0.07058,...,6.060991,6.060991,6.060991,6.060991,6.060991,6.060991,0.034718,0.034718,0.047950,0.047950


In [17]:
sub = test_data.loc[:,["ID","item_cnt_month2"]]
sub.rename(columns={"item_cnt_month2":"item_cnt_month"},inplace=True)
sub.to_csv("data/sub_xgboost.csv",index=False)